# EventHouse Schema Creation - RTI Solution Accelerator

## Schema Structure
- **Real-Time Events (3 main tables)**:
  - ClickstreamEvents: E-commerce user interactions, cart activities, purchase events
  - ManufacturingTelemetry: Equipment sensor data, temperature, vibration, defect rates  
  - ShippingEvents: Package tracking, delivery status, location updates
- **Reference Data (3 tables)**:
  - Sites: Manufacturing site master data
  - Assets: Equipment/machine reference data
  - Carriers: Shipping carrier information
- **Analytics (1 table)**:
  - AnomalyDetectionResults: Real-time anomaly detection results

## Automated Deployment
This notebook automatically creates all EventHouse tables, JSON ingestion mappings, streaming policies, and retention policies.
No manual copy-paste required - everything is executed programmatically.

In [ ]:
# This is already installed in Fabric 
#!pip install Kqlmagic --no-cache-dir --upgrade

In [ ]:
# STEP 1: Load Kqlmagic extension
print("🔧 Loading Kqlmagic extension...")
%reload_ext Kqlmagic
print("✅ Kqlmagic loaded successfully!")

In [ ]:
# STEP 2: Configuration and Connection
# 📋 REQUIRED: Get your connection details from Fabric first
# 
# HOW TO GET CONNECTION INFO:
# 1. Go to your Fabric workspace → fabrikam_eventhouse  
# 2. Expand 'Database details' and copy the 'Query URI'
# 3. The URI format is: https://<cluster>.kusto.fabric.microsoft.com/
# 4. Extract cluster name from the URI and update CLUSTER_NAME below

import sys
import json
import requests
from datetime import datetime

# EventHouse Configuration
# In Fabric, EventHouse and Database have the same name
EVENTHOUSE_NAME = "fabrikam_eventhouse"
CLUSTER_NAME = "trd-hvsyx3h9ej7989gfw7.z6"

print(f"📋 STEP 2: Connecting to EventHouse")
print(f"🏛️ Target EventHouse: {EVENTHOUSE_NAME}")
print(f"🖥️ Target Cluster: {CLUSTER_NAME}")

# Note: In Fabric, connection is handled automatically when you run KQL commands
print("✅ Configuration complete - ready to run KQL commands")
print("💡 Connection will be established automatically when running KQL cells")

In [ ]:
################################################################################################
# STEP 3: Connect to EventHouse and Validate Connection  
################################################################################################

# Connect to your EventHouse using the standard Fabric connection syntax
%kql kusto://code;cluster='trd-hvsyx3h9ej7989gfw7.z6.kusto.fabric.microsoft.com';database='fabrikam_eventhouse'

In [ ]:
################################################################################################
# STEP 4: Validate Connection Context
################################################################################################

%%kql 
.show database

In [ ]:
################################################################################################
# Real-Time Events Tables - Main data ingestion tables
################################################################################################

print("📊 Creating real-time events tables...")

In [ ]:
%%kql
.create table ClickstreamEvents (
    event_id: string,
    timestamp: datetime,
    event_type: string,
    user_id: string,
    session_id: string,
    sku: string,
    country: string,
    country_code: string,
    referral_source_type: string,
    referral_platform: string,
    product_id: string,
    spike_flag: bool,
    cart_spike_magnitude: int,
    client_info: dynamic,
    payload: dynamic
)

In [ ]:
%%kql
.create table ManufacturingTelemetry (
    event_type: string,
    timestamp: datetime,
    SiteId: string,
    City: string,
    AssetId: string,
    OperatorId: string,
    OperatorName: string,
    ProductId: string,
    SKU: string,
    BatchId: string,
    DefectProbability: real,
    Vibration: real,
    Temperature: real,
    Humidity: int
)

In [ ]:
%%kql
.create table ShippingEvents (
    event_id: string,
    timestamp: datetime,
    event_type: string,
    tracking_number: string,
    shipment_id: string,
    origin_location: string,
    destination_location: string,
    carrier: string,
    package_weight: real,
    estimated_delivery: datetime,
    actual_delivery: datetime,
    delay_minutes: int,
    status: string,
    location_coordinates: dynamic
)

In [ ]:
################################################################################################
# Reference Data Tables - Master data for lookups
################################################################################################

print("🗂️ Creating reference data tables...")

In [ ]:
%%kql
.create table Sites (
    SiteId: string,
    City: string,
    Country: string,
    Region: string,
    Latitude: real,
    Longitude: real
)

.create table Assets (
    AssetId: string,
    AssetName: string,
    AssetType: string,
    SiteId: string,
    InstallationDate: datetime,
    MaintenanceSchedule: string
)

.create table Carriers (
    CarrierId: string,
    CarrierName: string,
    ServiceLevel: string,
    Coverage_Area: string,
    AvgDeliveryTime: int
)

In [ ]:
%%kql
.create table AnomalyDetectionResults (
    timestamp: datetime,
    anomaly_type: string,
    source_table: string,
    entity_id: string,
    anomaly_score: real,
    baseline_value: real,
    current_value: real,
    severity: string,
    description: string,
    metadata: dynamic
)

In [ ]:
################################################################################################
# JSON Ingestion Mappings - Configure data ingestion from simulators
################################################################################################

print("🗂️ Creating JSON ingestion mappings...")

In [ ]:
%%kql
.create table ClickstreamEvents ingestion json mapping "ClickstreamMapping"
'['
'    {"column":"event_id","path":"$.event_id"},'
'    {"column":"timestamp","path":"$.timestamp"},'
'    {"column":"event_type","path":"$.event_type"},'
'    {"column":"user_id","path":"$.user_id"},'
'    {"column":"session_id","path":"$.session_id"},'
'    {"column":"sku","path":"$.sku"},'
'    {"column":"country","path":"$.country"},'
'    {"column":"country_code","path":"$.country_code"},'
'    {"column":"referral_source_type","path":"$.referral_source_type"},'
'    {"column":"referral_platform","path":"$.referral_platform"},'
'    {"column":"product_id","path":"$.product_id"},'
'    {"column":"spike_flag","path":"$.spike_flag"},'
'    {"column":"cart_spike_magnitude","path":"$.cart_spike_magnitude"},'
'    {"column":"client_info","path":"$.client_info"},'
'    {"column":"payload","path":"$.payload"}'
']'

In [ ]:
%%kql
.create table ManufacturingTelemetry ingestion json mapping "ManufacturingMapping"
'['
'    {"column":"event_type","path":"$.event_type"},'
'    {"column":"timestamp","path":"$.timestamp"},'
'    {"column":"SiteId","path":"$.SiteId"},'
'    {"column":"City","path":"$.City"},'
'    {"column":"AssetId","path":"$.AssetId"},'
'    {"column":"OperatorId","path":"$.OperatorId"},'
'    {"column":"OperatorName","path":"$.OperatorName"},'
'    {"column":"ProductId","path":"$.ProductId"},'
'    {"column":"SKU","path":"$.SKU"},'
'    {"column":"BatchId","path":"$.BatchId"},'
'    {"column":"DefectProbability","path":"$.DefectProbability"},'
'    {"column":"Vibration","path":"$.Vibration"},'
'    {"column":"Temperature","path":"$.Temperature"},'
'    {"column":"Humidity","path":"$.Humidity"}'
']'

In [ ]:
%%kql
.create table ShippingEvents ingestion json mapping "ShippingMapping"
'['
'    {"column":"event_id","path":"$.event_id"},'
'    {"column":"timestamp","path":"$.timestamp"},'
'    {"column":"event_type","path":"$.event_type"},'
'    {"column":"tracking_number","path":"$.tracking_number"},'
'    {"column":"shipment_id","path":"$.shipment_id"},'
'    {"column":"origin_location","path":"$.origin_location"},'
'    {"column":"destination_location","path":"$.destination_location"},'
'    {"column":"carrier","path":"$.carrier"},'
'    {"column":"package_weight","path":"$.package_weight"},'
'    {"column":"estimated_delivery","path":"$.estimated_delivery"},'
'    {"column":"actual_delivery","path":"$.actual_delivery"},'
'    {"column":"delay_minutes","path":"$.delay_minutes"},'
'    {"column":"status","path":"$.status"},'
'    {"column":"location_coordinates","path":"$.location_coordinates"}'
']'

In [ ]:
################################################################################################
# Streaming Ingestion Policies - Enable real-time data ingestion
################################################################################################

print("🌊 Enabling streaming ingestion policies...")

In [ ]:
%%kql
// Enable streaming ingestion for real-time data
.alter table ClickstreamEvents policy streamingingestion enable
.alter table ManufacturingTelemetry policy streamingingestion enable
.alter table ShippingEvents policy streamingingestion enable
.alter table AnomalyDetectionResults policy streamingingestion enable

In [ ]:
################################################################################################
# Data Retention Policies - Set appropriate retention periods
################################################################################################

print("📅 Setting data retention policies...")

In [ ]:
%%kql
// Clickstream: 30-day retention as per architecture
.alter table ClickstreamEvents policy retention '{"SoftDeletePeriod": "30.00:00:00", "Recoverability": "Enabled"}'

// Manufacturing & Shipping: 90-day retention as per architecture  
.alter table ManufacturingTelemetry policy retention '{"SoftDeletePeriod": "90.00:00:00", "Recoverability": "Enabled"}'
.alter table ShippingEvents policy retention '{"SoftDeletePeriod": "90.00:00:00", "Recoverability": "Enabled"}'

// Anomaly Results: 12-month retention as per architecture
.alter table AnomalyDetectionResults policy retention '{"SoftDeletePeriod": "365.00:00:00", "Recoverability": "Enabled"}'

// Reference tables: Long-term retention
.alter table Sites policy retention '{"SoftDeletePeriod": "3650.00:00:00", "Recoverability": "Enabled"}'
.alter table Assets policy retention '{"SoftDeletePeriod": "3650.00:00:00", "Recoverability": "Enabled"}'
.alter table Carriers policy retention '{"SoftDeletePeriod": "3650.00:00:00", "Recoverability": "Enabled"}'

In [ ]:
################################################################################################
# Deployment Verification - Validate all objects were created successfully
################################################################################################

print("🔍 Running deployment verification...")

In [ ]:
%%kql
.show tables

In [ ]:
%%kql
.show table ClickstreamEvents ingestion json mappings
.show table ManufacturingTelemetry ingestion json mappings
.show table ShippingEvents ingestion json mappings

In [ ]:
%%kql
.show tables | count

In [ ]:
# Final Success Confirmation
print("🎉 SUCCESS: EventHouse schema deployment completed!")
print(f"📊 EventHouse: {EVENTHOUSE_NAME}")
print("🗃️ Tables: 7 created (3 events + 3 reference + 1 analytics)")
print("🗂️ Mappings: 3 JSON ingestion mappings configured")
print("🌊 Policies: Streaming ingestion enabled")
print("📅 Retention: Configured per architecture (30d/90d/12m)")
print("")
print("📋 NEXT STEPS:")
print("1. Set up EventStream to connect simulators")
print("2. Configure Activator rules for alerts")
print("3. Create RTI dashboards")
print("4. Start data simulators")